<a href="https://colab.research.google.com/github/MHKhan18/colab-notebooks/blob/main/Lecture4_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

We will look at an example of the transportation problem. There are street with blocks numbered 1 to n. Walking from s to s + 1 takes 1 minute. Taking a magic tram from s to 2s takes 2 minutes. How to travel from 1 to n in the least time?

To formalize this problem, first we define the start state, successor and cost function. 

In [ ]:
import heapq, collections, re, sys, time, os, random

# Data structure for supporting uniform cost search.
class PriorityQueue:
    def  __init__(self):
        self.DONE = -100000
        self.heap = []
        self.priorities = {}  # Map from state to priority

    # Insert |state| into the heap with priority |newPriority| if
    # |state| isn't in the heap or |newPriority| is smaller than the existing
    # priority.
    # Return whether the priority queue was updated.
    def update(self, state, newPriority, newHistory):
        oldPriority = self.priorities.get(state)
        if oldPriority == None or newPriority < oldPriority:
            self.priorities[state] = newPriority
            heapq.heappush(self.heap, (newPriority, state, newHistory))
            return True
        return False

    # Returns (state with minimum priority, priority)
    # or (None, None) if the priority queue is empty.
    def removeMin(self):
        while len(self.heap) > 0:
            priority, state, history = heapq.heappop(self.heap)
            if self.priorities[state] == self.DONE: continue  # Outdated priority, skip
            self.priorities[state] = self.DONE
            return (state, priority, history)
        return (None, None, None) # Nothing left...


In [ ]:
class TransportationProblem(object):
    def __init__(self, N):
        # N = number of blocks
        self.N = N
    def startState(self):
        return 1
    def succAndCost(self, state):
        # return list of (action, newState, cost) triples
        result = []
        if state + 1 <= self.N:
          result.append(('walk', state+1, 1))
        if state * 2 <= self.N:
          result.append(('tram', state*2, 2))
        return result
    def isEnd(self, state):
        return state == self.N

In [ ]:
def backtrackingSearch(problem):
    # TODO: Best solution found so far (dictionary because of python scoping technicality)
    best = {
        'cost': float('+inf'),
        'history': None
    }
    def recurse(state, history, totalCost):
        if problem.isEnd(state):
            # Update the best solution so far
            # TODO: Update the best solution so far
            if totalCost < best['cost']:
              best['cost'] = totalCost
              best['history'] = history
            return
        # TODO: Recurse on children
        for action, newState, cost in problem.succAndCost(state):
          recurse(newState, history+[(action, newState, cost)], totalCost + cost)
    # TODO: start w/ start state, call it on the start state
    recurse(problem.startState(), history=[], totalCost=0)
    return (best['cost'], best['history'])

In [ ]:
def dynamicProgramming(problem):
    cache = {} # state -> futureCost(state)
    def futureCost(state):
        # Base case
        if problem.isEnd(state):
            return 0
        if state in cache:
            return cache[state]
        result = min(cost + futureCost(newState) \
          for action, newState, cost in problem.succAndCost(state))
        cache[state] = result
        return result
    return (futureCost(problem.startState()), [])

In [ ]:
def uniformCostSearch(problem):
    frontier = PriorityQueue()
    frontier.update(problem.startState(), 0, [])
    explored = set([])
    while True:
        # TODO: Pop the minimum cost state from frontier and add to explored
        state, pastCost, history = frontier.removeMin()
        explored.add(state)
        if problem.isEnd(state):
            # TODO: return the past cost and history
            return (pastCost, history)
        # TODO: iterate from successor state
        # for each successor state, if it's not explored, update the cost and history
        for action, newState, cost in problem.succAndCost(state):
            if newState in explored:
               continue
            frontier.update(newState, pastCost + cost, history + [(action, newState, cost)])

In [ ]:
def printSolution(solution):
    totalCost, history = solution
    print('totalCost: {}'.format(totalCost))
    for item in history:
        print(item)

In [ ]:
problem = TransportationProblem(N=100)
#print(problem.succAndCost(3))
printSolution(uniformCostSearch(problem))

totalCost: 13
('walk', 2, 1)
('walk', 3, 1)
('tram', 6, 2)
('tram', 12, 2)
('tram', 24, 2)
('walk', 25, 1)
('tram', 50, 2)
('tram', 100, 2)
